In [14]:
! pip install datasets transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [15]:
from transformers.utils import send_example_telemetry

send_example_telemetry("tokenizer_training_notebook", framework="none")

# Train your own Tokenizeer on existing tokenizer
 Why do we need to train ?
 * Transformer models very often use subword tokenization algorithms, and they need to be trained to identify the parts of words that are often present in the corpus you are using

In [16]:
# Getting a corpus -collection of words
from datasets import load_dataset
dataset=load_dataset('wikitext',name='wikitext-2-raw-v1',split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [17]:
dataset

Dataset({
    features: ['text'],
    num_rows: 36718
})

In [18]:
dataset[1]

{'text': ' = Valkyria Chronicles III = \n'}

In [19]:
batch_size=1000
all_texts=[dataset[i:i+batch_size]['text'] for i in range(0, len(dataset),batch_size)]

In [20]:
def batch_iterator():
  for i in range(0,len(dataset),batch_size):
    # yield- allows function to preduce value and pause execution.
    yield dataset[i:i+batch_size]['text']

In [21]:
# Using an existing tokenizer
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [22]:
tokenizer.is_fast

True

In [23]:
# We feed the training corpus to  the train_new_from_iterator
new_tokenizer=tokenizer.train_new_from_iterator(batch_iterator(),vocab_size=25000)

In [24]:
new_tokenizer(dataset[:5]['text'])

{'input_ids': [[], [301, 8639, 9504, 3050, 301, 315], [], [4720, 74, 4825, 889, 8639, 491, 529, 672, 6944, 475, 267, 9504, 374, 2809, 529, 10879, 231, 100, 162, 255, 113, 14391, 4046, 113, 4509, 95, 18351, 4509, 256, 4046, 99, 4046, 22234, 96, 19, 264, 6437, 272, 8639, 281, 261, 3518, 2035, 491, 373, 264, 5162, 3305, 290, 344, 8639, 9504, 3050, 2616, 1822, 264, 364, 259, 14059, 1559, 340, 2393, 1527, 737, 1961, 370, 805, 3604, 288, 7577, 14, 54, 782, 337, 261, 4840, 15585, 272, 19958, 284, 1404, 1696, 284, 1822, 264, 385, 364, 261, 1431, 737, 284, 261, 8639, 906, 272, 2531, 1858, 286, 261, 1112, 9658, 281, 14059, 288, 1626, 340, 645, 6556, 344, 520, 14434, 264, 261, 1485, 3436, 7515, 290, 261, 518, 737, 288, 4750, 261, 302, 22039, 302, 264, 259, 21720, 1743, 3836, 5654, 261, 4259, 281, 4742, 490, 724, 261, 3581, 1351, 283, 1114, 579, 952, 4010, 1985, 2563, 288, 453, 2128, 807, 935, 261, 7655, 3836, 302, 2038, 314, 271, 89, 22414, 302, 272, 315], [324, 737, 1022, 1984, 284, 1525, 264, 7

In [25]:
# You can save it as well
new_tokenizer.save_pretrained('my-new-tokenizer')

('my-new-tokenizer/tokenizer_config.json',
 'my-new-tokenizer/special_tokens_map.json',
 'my-new-tokenizer/vocab.json',
 'my-new-tokenizer/merges.txt',
 'my-new-tokenizer/added_tokens.json',
 'my-new-tokenizer/tokenizer.json')

In [ ]:
# You can push it to your HuggingFace hub as well
from huggingface_hub import notebook_login

notebook_login()

In [27]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
# Push your tokenizer to the HuggingFace Hub
new_tokenizer.push_to_hub("my-new-shiny-tokenizer")

In [29]:
# Load you own tokenizer
tok = new_tokenizer.from_pretrained("my-new-tokenizer")

In [30]:
# Fetching your own Tokenizer from HuggingFace Hub
tok = new_tokenizer.from_pretrained("OmBhandwalkar/my-new-shiny-tokenizer")

tokenizer_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/396k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

# Building the tokenizer from scratch
Before tokenize words into tensors, there are several steps done by the tokenizers library.
1. Normalization-
* Lowercase the text
* Remove spaces or special characters
* Apply unicode formatting if needed.

2. Pre-Tokenization-
* Split the text on space.

3. Model-
* Handles all the sub-tokens discovery and generation.The trainable part that turns pieces into tokens.

4. Post-Processing-
* Provides advanced construction features to be compatible with some of the transformers model.
* Adds extra tokes needed for certain models .
* ex- BERT adds [CLS] at the begining and [SEP] between sentence.

5. Decoding(reverse process)-
* Turn tokens back into human-readable text.

# Will use the `Trainer` class from Hugging Face to train your tokenizer on your data.

Creating a `Tokenizer` with an empty `wordPiece` model

In [31]:
from tokenizers import decoders,models,normalizers,pre_tokenizers,processors,trainers,Tokenizer
tokenizer=Tokenizer(models.WordPiece(unk_token='[UNK]'))

Now we are applying the several steps before tokenize it.

In [32]:
# Lowercasing the text
tokenizer.normalizer=normalizers.BertNormalizer(lowercase=True)

In [33]:
# You can customize your own block.
tokenizer.normalizer=normalizers.Sequence([
    # It splits accented characters into base character + accent
    normalizers.NFD(),
    # Converts all latters into lowercase.
    normalizers.Lowercase(),
    # Remove accents after NFD split
    normalizers.StripAccents()
])

We also have `BERTPreTokenizer`  it pretokenize the text on the basis of white spaces and punctuation

In [34]:
tokenizer.pre_tokenizer=pre_tokenizers.BertPreTokenizer()

In [35]:
# Pre-tokenizer split raw text into smaller parts. keeps track of where  each piece is in the original text.
tokenizer.pre_tokenizer.pre_tokenize_str('This is an example')

[('This', (0, 4)), ('is', (5, 7)), ('an', (8, 10)), ('example', (11, 18))]

Now we train our tokenizer by using `WordPieceTrainer`. We are also passing special tokens to the trainer.

In [36]:
# [UNK]-Unknown token
# [PAD]-Padding token
# [CLS]-Classification token-> marks start of input (used by BERT)
# [SEP]-Separator token -> separates sentence pairs
# [MASK]-Mask token -> used in masked language modelin
special_tokens=['[UNK]','[PAD]','[CLS]','[SEP]','[MASK]']
trainer=trainers.WordPieceTrainer(vocab_size=25000,special_tokens=special_tokens)

In [37]:
# We add some text to train
tokenizer.train_from_iterator(batch_iterator(),trainer=trainer)

 Now tokenizer is trained. We define post-processor,we need to add [CLS] token at the begining and [SEP] at the ending. We use `TemplateProcessing` to do this, which requires to know IDs of CLS and SEP tokens

In [39]:
cls_token_id=tokenizer.token_to_id('[CLS]')
sep_token_id=tokenizer.token_to_id('[SEP]')
print(cls_token_id,sep_token_id)

2 3


And here is how we can build our post processor. We have to indicate in the template how to organize the special tokens with one sentence (`$A`) or two sentences (`$A` and `$B`). The `:` followed by a number indicates the token type ID to give to each part.

In [40]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [42]:
encoding=tokenizer.encode("This is one sentence.", "With this one we have a pair.")
encoding.tokens

['[CLS]',
 'this',
 'is',
 'one',
 'sentence',
 '.',
 '[SEP]',
 'with',
 'this',
 'one',
 'we',
 'have',
 'a',
 'pair',
 '.',
 '[SEP]']

In [43]:
encoding.type_ids

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [44]:
tokenizer.decoder = decoders.WordPiece(prefix="##")


Now that our tokenizer is finished, we need to wrap it inside a Transformers object to be able to use it with the Transformers library. More specifically, we have to put it inside the class of tokenizer fast corresponding to the model we want to use, here a BertTokenizerFast

In [45]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

# BPE model like GPT-2
Let's now have a look at how we can create a BPE tokenizer like the one used for training GPT-2. The first step is to create a Tokenizer with an empty BPE model:

In [46]:
tokenizer = Tokenizer(models.BPE())

Like before, we have to add the optional normalization (not used in the case of GPT-2) and we need to specify a pre-tokenizer before training. In the case of GPT-2, the pre-tokenizer used is a byte level pre-tokenizer:

In [47]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [48]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

In [ ]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [ ]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [ ]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)